# YAML Model

In [33]:
import ZODB
from Products.Five.browser.tests.pages import SimpleView
from Testing.makerequest import makerequest 							# makerequest(context)
from Testing.ZopeTestCase.testZODBCompat import make_request_response 	# make_request_response()[1]
from Acquisition import aq_get

from Products.zms import standard
from Products.zms import rest_api

import os
import re
import shutil
import tempfile
import urllib
import json
import requests
import yaml

## Step-2: Open ZODB database

In [ ]:
# db.close()
# Create a ZODB connection to an existing ZODB database file
# Important: Fit working directory to your Zope instance
try:
	wd = '/home/zope/instance/zms5_dev/var'
	db = ZODB.DB(os.path.join(wd, 'Data.fs'))
	conn = db.open()
	root = conn.root
	###{'Application': <Application at >}
except:
	db.close()
	print('Error: Database connection had to be closed before reopened.')

## Step-3: Get ZODB Context

In [36]:
# ZMS-Node /myzmsx/content
# ##############################################
context = root.Application.myzmsx.content 
# ##############################################

# Add REQUEST to zmscontext object
context = makerequest(context)
# Add REQUEST vars
context.REQUEST.environ.setdefault('SERVER_NAME','localhost')
context.REQUEST.environ.setdefault('SERVER_PORT', '8092')
context.REQUEST['URL' ]= 'http://localhost:8092'
context.REQUEST.set('lang','ger')
context.REQUEST.set('path_to_handle','')
# Add RESPONSE
context.REQUEST.set('RESPONSE', make_request_response()[1])

zmscontext = context

In [95]:
data = zmscontext.getMetaobjManager().getMetaobj('ZMSFlexbox')['Attrs'][2]

In [98]:
data['custom']

'@media only screen and (min-width: 576px) {\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) {\n\t\tdisplay: flex;\n\t\tflex-direction: row;\n\t\tjustify-content: space-between;\n\t}\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) .ZMSFlexboxItem {\n\t\tflex-direction: column;\n\t\tbackground: rgba(0,0,0,.05);\n\t\tpadding: 1rem;\n\t\tmargin:1rem 0;\n\t}\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) .ZMSFlexboxItem > * {\n\t\tmargin:0 !important;\n\t\tpadding:0 !important;\n\t}\n\tbody:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem {\n\t\tflex-basis:calc((100% / var(--data-itemcount)) - 3rem);\n\t}\n\tbody:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem .ZMSGraphic .graphic,\n\tbody:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem .ZMSGraphic .graphic img {\n\t\twidth:100% !important;\n\t\tmax-width:100% !important;\n\t}\n\tbody:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem .ZMSGraphic .text {\n\t\tpadding-top:1em;\n\t}\n}\n}'

In [99]:
import yaml

# Step 1: Define custom string type
class LiteralStr(str): pass

# Step 2: Register representer for both LiteralStr and str
def literal_str_representer(dumper, data):
    return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')

yaml.add_representer(LiteralStr, literal_str_representer)
yaml.add_representer(str, yaml.representer.SafeRepresenter.represent_str)  # fallback for normal strings

# Step 3: Decode escaped string into real multiline string
raw_str = data['custom'].encode().decode('unicode_escape')

# Step 4: Wrap with LiteralStr
data['custom'] = LiteralStr(raw_str)

# Step 5: Dump YAML
yaml_output = yaml.dump(data, sort_keys=False, allow_unicode=True)
print(yaml_output)


custom: "@media only screen and (min-width: 576px) {\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable)\
  \ {\n\t\tdisplay: flex;\n\t\tflex-direction: row;\n\t\tjustify-content: space-between;\n\
  \t}\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) .ZMSFlexboxItem {\n\t\t\
  flex-direction: column;\n\t\tbackground: rgba(0,0,0,.05);\n\t\tpadding: 1rem;\n\t\
  \tmargin:1rem 0;\n\t}\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) .ZMSFlexboxItem\
  \ > * {\n\t\tmargin:0 !important;\n\t\tpadding:0 !important;\n\t}\n\tbody:not(.zmi)\
  \ .ZMSFlexbox .ZMSFlexboxItem {\n\t\tflex-basis:calc((100% / var(--data-itemcount))\
  \ - 3rem);\n\t}\n\tbody:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem .ZMSGraphic .graphic,\n\
  \tbody:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem .ZMSGraphic .graphic img {\n\t\twidth:100%\
  \ !important;\n\t\tmax-width:100% !important;\n\t}\n\tbody:not(.zmi) .ZMSFlexbox\
  \ .ZMSFlexboxItem .ZMSGraphic .text {\n\t\tpadding-top:1em;\n\t}\n}\n}"
default: ''
id: f_css_default